# Import required library package , EDA and Data preprocessing

In [ ]:
import sklearn
import os
import numpy as np
import pandas as pd
#libraries needed

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from sklearn import tree
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, recall_score, precision_score


import matplotlib.pyplot as plt
import seaborn as sns

import graphviz

In [ ]:
sklearn.__version__

In [ ]:
os.chdir('../input/higs-bonsons-and-background-process')

In [ ]:
data = pd.read_csv("train.csv",na_values="?")

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
data.head(10)

In [ ]:
data.tail(10)

In [ ]:
data.nunique()

In [ ]:
data.dtypes

In [ ]:
#data.describe()
data.describe()

In [ ]:
data.isnull().sum()

# Define categorical and numerical columns

In [ ]:
cat_cols = data.columns[data.nunique() <= 5]
cat_cols

In [ ]:
num_cols = data.columns[data.nunique() > 5]
num_cols

In [ ]:
data.dropna(axis=0, inplace=True)
#row=0,col=1 for dropping null values

In [ ]:
data = data[data['class'].notna()]

In [ ]:
data.columns

In [ ]:
#we are removing ID since it has all the unique values which means number of rows(68636) is equal to number of unique values of ID
data.drop(["id"], axis = 1, inplace=True)
num_cols = data.columns[data.nunique() > 5]

In [ ]:
data.head()

In [ ]:
data.isnull().sum()

In [ ]:
data.nunique()

# Define X & y

In [ ]:
y=data.loc[:, "class"]
y

In [ ]:
X= data.loc[:,data.columns!='class']
X

In [ ]:
print(X.shape, y.shape)

# Train Test Split

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=123)

In [ ]:
num_imputer = SimpleImputer(strategy='mean')
num_imputer = num_imputer.fit(X_train[num_cols])

In [ ]:
X_train_num = num_imputer.transform(X_train[num_cols])
X_train_num = pd.DataFrame(X_train_num, columns=num_cols)

In [ ]:
X_test_num = num_imputer.transform(X_test[num_cols])
X_test_num = pd.DataFrame(X_test_num, columns=num_cols)

In [ ]:
type(cat_cols)

In [ ]:
cat_cols=cat_cols.drop('class')

In [ ]:
cat_cols

In [ ]:
#perform imputation on categorical columns
cat_imputer = SimpleImputer(strategy='most_frequent')
cat_imputer = cat_imputer.fit(X_train[cat_cols])

In [ ]:
X_train_cat = cat_imputer.transform(X_train[cat_cols])
X_train_cat = pd.DataFrame(X_train_cat, columns= cat_cols)

In [ ]:
X_test_cat = cat_imputer.transform(X_test[cat_cols])
X_test_cat = pd.DataFrame(X_test_cat, columns= cat_cols)

In [ ]:
#do onehotencoding on categorical columns###always drop either first or last column=onehotencoder
onehotencoder = OneHotEncoder(handle_unknown='error',drop='first')
onehotencoder = onehotencoder.fit(X_train_cat)
ohe_cat_col_names = onehotencoder.get_feature_names(cat_cols)

In [ ]:
X_train_cat_onehotencoded = onehotencoder.transform(X_train_cat).toarray()
X_train_cat_onehotencoded = pd.DataFrame(X_train_cat_onehotencoded, columns=ohe_cat_col_names)

In [ ]:
X_test_cat_onehotencoded = onehotencoder.transform(X_test_cat).toarray()
X_test_cat_onehotencoded = pd.DataFrame(X_test_cat_onehotencoded, columns=ohe_cat_col_names)

In [ ]:
X_train_cat

In [ ]:
X_train_cat['jet1b.tag'].nunique()

# Implementing onehotencoder

In [ ]:
X_train_cat_onehotencoded

In [ ]:
#merge the numerical and categorical columns
X_train = pd.concat([X_train_num, X_train_cat_onehotencoded], axis=1)

X_test = pd.concat([X_test_num, X_test_cat_onehotencoded], axis=1)


# Decision Tree

In [ ]:
#build DecisionTree
from sklearn.tree import DecisionTreeClassifier
clf_dt = DecisionTreeClassifier()
dt_param_grid = {
    'criterion': ['entropy', 'gini'],
    'max_depth': [6,8,10,12], 
    'min_samples_split': [2, 10, 20],
    'min_samples_leaf': [1, 5, 10],
    'class_weight':['balanced']
}
dt_grid_bal = GridSearchCV(clf_dt, param_grid=dt_param_grid, cv=5, n_jobs=-1)

In [ ]:
%%time
dt_grid_bal.fit(X_train,y_train)
#fitting dt_grid_bal on train_data

In [ ]:
dt_grid_bal.best_estimator_

In [ ]:
train_predictions = dt_grid_bal.predict(X_train)
test_predictions = dt_grid_bal.predict(X_test)

In [ ]:
### Train data accuracy
from sklearn.metrics import accuracy_score,f1_score

print("TRAIN Conf Matrix : \n", confusion_matrix(y_train, train_predictions))
print("\nTRAIN DATA ACCURACY",accuracy_score(y_train,train_predictions))
print("\nTrain data f1-score for class '1'",f1_score(y_train,train_predictions,pos_label=1))
print("\nTrain data f1-score for class '2'",f1_score(y_train,train_predictions,pos_label=0))

### Test data accuracy
print("\n\n--------------------------------------\n\n")

print("TEST Conf Matrix : \n", confusion_matrix(y_test, test_predictions))
print("\nTEST DATA ACCURACY",accuracy_score(y_test,test_predictions))
print("\nTest data f1-score for class '1'",f1_score(y_test,test_predictions,pos_label=1))
print("\nTest data f1-score for class '2'",f1_score(y_test,test_predictions,pos_label=0))

In [ ]:
unseendata=pd.read_csv('test.csv', na_values=["?", "", "null"])

In [ ]:
unseendata_ids = unseendata.loc[:, "id"]
unseendata.drop(["id"], axis = 1, inplace=True)

In [ ]:
unseendata.loc[29413, :]

In [ ]:
X_unseendata=unseendata.loc[:,unseendata.columns!='class']

In [ ]:
y_unseendata=unseendata.loc[:, "class"]

In [ ]:
X_unseendata_num=num_imputer.transform(X_unseendata[num_cols])
X_unseendata_num=pd.DataFrame(X_unseendata_num, columns=num_cols)
X_unseendata_cat = cat_imputer.transform(X_unseendata[cat_cols])
X_unseendata_cat = pd.DataFrame(X_unseendata_cat, columns=cat_cols)

In [ ]:
X_unseendata_num.isnull().sum()

In [ ]:
X_unseendata_cat_onehotencoded = onehotencoder.transform(X_unseendata_cat).toarray()
X_unseendata_cat_onehotencoded = pd.DataFrame(X_unseendata_cat_onehotencoded, columns=ohe_cat_col_names)

In [ ]:
X_unseendata = pd.concat([X_unseendata_num, X_unseendata_cat_onehotencoded], axis=1)
X_unseendata

In [ ]:
unseendata_predictions = dt_grid_bal.predict(X_unseendata)

In [ ]:
### unseendata data accuracy
print("\n\n--------------------------------------\n\n")

print("unseendata Conf Matrix : \n", confusion_matrix(y_unseendata, unseendata_predictions))
print("\nunseendata DATA ACCURACY",accuracy_score(y_unseendata,unseendata_predictions))
print("\nunseendata data f1-score for class '1'",f1_score(y_unseendata,unseendata_predictions,pos_label=1))
print("\nunseendata data f1-score for class '2'",f1_score(y_unseendata,unseendata_predictions,pos_label=0))

In [ ]:
final_outputdt=pd.DataFrame({'id':unseendata_ids, 'prediction':unseendata_predictions})
id_n_prediction = ["id", "prediction"]
final_outputdt = final_outputdt.loc[:,id_n_prediction]
final_outputdt.to_csv("/kaggle/working/final_outputdt.csv")
final_outputdt

In [ ]:
x = pd.read_csv("/kaggle/working/final_outputdt.csv")

In [ ]:
x.shape